In [1]:
# Data Preprocessing
'''
1 - Loading Audio Data: We use the librosa library to load the audio files. It's important to specify the sample rate (sr) as None to preserve the original sampling rate.

2 - Converting to Mel Spectrogram: Instead of using raw audio data, we convert it into a Mel spectrogram. A Mel spectrogram is a visual representation of audio data that’s easier for a neural network to process. We create these spectrograms using librosa.

3 - Resizing Spectrograms: To ensure that all spectrograms have a consistent shape, we resize them to a target shape (e.g., 128x128 pixels). This step is crucial to ensure compatibility with the neural network.
'''

"\n1 - Loading Audio Data: We use the librosa library to load the audio files. It's important to specify the sample rate (sr) as None to preserve the original sampling rate.\n\n2 - Converting to Mel Spectrogram: Instead of using raw audio data, we convert it into a Mel spectrogram. A Mel spectrogram is a visual representation of audio data that’s easier for a neural network to process. We create these spectrograms using librosa.\n\n3 - Resizing Spectrograms: To ensure that all spectrograms have a consistent shape, we resize them to a target shape (e.g., 128x128 pixels). This step is crucial to ensure compatibility with the neural network.\n"

In [2]:
import os
import librosa
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [3]:
# Model Architecture
''' 
Our model architecture consists of the following layers:

- Input layer with the shape of our resized spectrograms.
- Two convolutional layers with ReLU activation functions followed by max-pooling layers.
- A flatten layer to prepare the data for the fully connected layers.
- A dense hidden layer with ReLU activation.
- An output layer with as many neurons as there are classes (in our case, two: “cat” and “dog”) and a softmax activation function to produce class probabilities.
'''

' \nOur model architecture consists of the following layers:\n\n- Input layer with the shape of our resized spectrograms.\n- Two convolutional layers with ReLU activation functions followed by max-pooling layers.\n- A flatten layer to prepare the data for the fully connected layers.\n- A dense hidden layer with ReLU activation.\n- An output layer with as many neurons as there are classes (in our case, two: “cat” and “dog”) and a softmax activation function to produce class probabilities.\n'

In [4]:
data_dir = '../datasets/training_data'
classes = ['cat', 'dog']

# load and pre-process audio data
def load_and_preprocess_data(data_dir, classes, target_shape = (128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr = None)

                # perform pre-processing (e.g. convert to Mel spectogram and resize)
                mel_spectogram = librosa.feature.melspectrogram(y = audio_data, sr = sample_rate)
                mel_spectogram = resize(np.expand_dims(mel_spectogram, axis =- 1), target_shape)
                data.append(mel_spectogram)
                labels.append(i)
    return np.array(data), np.array(labels)

# split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)

# convert labels to one-hot encoding and splitting test and train
labels = to_categorical(labels, num_classes=len(classes))
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [5]:
# Building the Model
'''
Now that we have preprocessed our data, we can build our deep learning model. In our case, we create a convolutional neural network (CNN), which is a common choice for image and spectrogram-based tasks.
'''

'\nNow that we have preprocessed our data, we can build our deep learning model. In our case, we create a convolutional neural network (CNN), which is a common choice for image and spectrogram-based tasks.\n'

In [6]:
# create a neural network model
input_shape = x_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu')(input_layer)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [7]:
# Compiling the Model
'''
We compile the model using the Adam optimizer and categorical cross-entropy loss function. The accuracy metric is used to monitor the model’s performance during training.
'''

'\nWe compile the model using the Adam optimizer and categorical cross-entropy loss function. The accuracy metric is used to monitor the model’s performance during training.\n'

In [8]:
# compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# training the model
'''
Key Training Parameters: 
- We train the model for 200 epochs with a batch size of 32. The number of epochs represents how many times the model will see the entire training dataset.
- We monitor and report training metrics such as loss and accuracy.
'''
model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/200


6/6 [==============================] - 2s 236ms/step - loss: 20.6525 - accuracy: 0.6845 - val_loss: 3.3395 - val_accuracy: 0.8571
Epoch 2/200
6/6 [==============================] - 1s 232ms/step - loss: 2.4325 - accuracy: 0.9226 - val_loss: 0.4041 - val_accuracy: 0.8810
Epoch 3/200
6/6 [==============================] - 1s 199ms/step - loss: 0.4171 - accuracy: 0.9167 - val_loss: 0.3430 - val_accuracy: 0.8810
Epoch 4/200
6/6 [==============================] - 1s 216ms/step - loss: 0.1776 - accuracy: 0.9524 - val_loss: 0.2159 - val_accuracy: 0.9048
Epoch 5/200
6/6 [==============================] - 1s 206ms/step - loss: 0.1002 - accuracy: 0.9881 - val_loss: 0.1351 - val_accuracy: 0.9286
Epoch 6/200
6/6 [==============================] - 1s 197ms/step - loss: 0.0570 - accuracy: 1.0000 - val_loss: 0.1164 - val_accuracy: 0.9524
Epoch 7/200
6/6 [==============================] - 1s 211ms/step - loss: 0.0516 - accuracy: 0.9821 - val_loss: 0.1364 - val_accuracy: 0.9524
Epoch 8/20

In [9]:
# model evaluation
test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

0.9523809552192688


In [10]:
# Saving and Loading the Model
'''
Once our model is trained and evaluated, we save it to disk using the HDF5 format. This allows us to load and use the model for future predictions without having to retrain it each time.
'''
model.save('audio_classification_model.h5')

C:\Users\DEEP\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Testing the Model on New Audio
'''
We provide a function to test the trained model on new audio files. This function loads an audio file, preprocesses it, makes predictions, and displays the predicted class along with class probabilities. This is a crucial step in applying the model to real-world scenarios.
'''

'\nWe provide a function to test the trained model on new audio files. This function loads an audio file, preprocesses it, makes predictions, and displays the predicted class along with class probabilities. This is a crucial step in applying the model to real-world scenarios.\n'

In [12]:
# load the saved model
model = load_model('audio_classification_model.h5')

# define the target shape for input spectograms
target_shape = (128, 128)

# define file_path and class labels
file_path = '../datasets/testing_data'
classes = ['cat', 'dog']

# function to pre-process and classify ans audio file
def test_audio(file_path, model):
    # load and pre-process the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram =tf.reshape(mel_spectrogram, (1,)+target_shape+(1,))

    # make predictions
    predictions = model.predict(mel_spectrogram)

    # get the class probabilities
    class_probabilities = predictions[0]

    # get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    return class_probabilities, predicted_class_index

In [13]:
# testing-1
test_audio_file = 'test1.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# displaying results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio-1 is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 95ms/step
Class: cat, Probability: 0.0000
Class: dog, Probability: 1.0000
The audio-1 is classified as: dog
Accuracy: 1.0000


In [14]:
# testing-2
test_audio_file = 'test2.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# displaying results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio-2 is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 27ms/step
Class: cat, Probability: 1.0000
Class: dog, Probability: 0.0000
The audio-2 is classified as: cat
Accuracy: 1.0000
